## Import

In [1]:
import ROOT as rt
import csv
import re
import sys
import os
import collections
from collections import OrderedDict
import uproot
import pandas as pd
from root_numpy import array2tree
import scipy
import awkward
import numpy as np
import time
import math
from scipy.stats import norm
#import PrettyTable

sys.path.append('/afs/desy.de/user/l/lbenato/cms-lpc-llp_repo/run3_muon_system_analysis/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot, addOverflow
from helper_functions import deltaR, deltaPhi, drawCMS
#from helper import *

rt.gStyle.SetOptStat(0)
rt.gROOT.SetBatch(True)

Welcome to JupyROOT 6.24/06


## Load ntuples

## Load variables

In [184]:
fpath =OrderedDict()
tree = OrderedDict()

start_t = time.time()
data_year = 2022

tag  = 'V1p19'
vers = 'v6'
#v1 includes events without selections
#v3 includes events with at least 2 clusters
#v4: no event level cuts, clusters made with DBSCAN, HLT time definition
#v5: no event level cuts, clusters made with CA

data_path = '/nfs/dust/cms/group/cms-llp/muon_system_run3/'+tag+'/Data'+str(data_year)+'/'+vers+'/'
lumi = 23.02*1000
xsec = 48.

if data_year == 2022:
    #fpath['data'] = data_path + 'DisplacedJet-EXOCSCCluster_Run2022EFG-PromptReco-v1_goodLumi.root'
    fpath['signal'] = data_path + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted.root'

NEvents = {}
for k,v in fpath.items():
    root_dir = uproot.open(v)
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    a = tree[k]["weight"].array()
    #print(a)
    if k=='signal':
        signal_gen_weight = tree[k]["weight"].array()

signal_gen_yield = np.sum(signal_gen_weight)*lumi
print(signal_gen_yield)

for k, T in tree.items():
    branch_names = T.keys()

1118331.3125610352


In [185]:
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
ring_cut = 50

#cut_based = True
#cut_based_version = 'v4'

intime = True
DPHI_CUT = 1


gLLP_csc = {}


selections_cluster = {}
sel_cluster = {}
sel_jetveto_csc = {}
sel_muonveto_csc = {}
sel_jetveto_dt = {}
sel_muonveto_dt = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}
genJetPt = {}
genJetPhi = {}
genMet = {}
genMetPhi = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
nCscRings = {}
nDtRings = {}

weight = {}
all_weight = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
cscClusterTimeSpread = {}
bdt_score = {}
nCscChambers = {}

sel_ev = {}
sel_ev_post = {}
hlt_sel_ev = {}
cluster_index = ''
nRings = {}

#CSC cluster variables
cscClusterMuonVetoPt = {}
cscClusterJetVetoPt = {}
cscClusterPhi = {}
cscClusterEta = {}
cscClusterEta2 = {}
cscClusterEta3 = {}
cscClusterR = {}
cscClusterZ = {}
cscClusterSize = {}
cscClusterSize2 = {}
cscClusterSize3 = {}
cscClusterTimeSpread = {}
cscClusterTime = {}
cscClusterTime2 = {}
cscClusterTime3 = {}
cscClusterMet_dPhi = {}

cscClusterNStation = {}
cscClusterDphi = {}
nCscClusters = {}

cscClusterNRechitMinus11 = {}
cscClusterNRechitPlus11 = {}
cscClusterNRechitMinus12 = {}
cscClusterNRechitPlus12 = {}


#DT cluster variables
dtClusterMuonVetoPt = {}
dtClusterJetVetoPt = {}
dtClusterTime = {}
dtClusterPhi = {}
dtClusterEta = {}
dtClusterSize = {}
dtClusterDphi = {}
nDtClusters = {}
dtClusterNStation = {}
dtClusterMaxStation = {}
dtClusterNHitStation1 = {}
dtCluster_match_MB1hits_0p4 = {}
dtCluster_match_MB1hits_0p5 = {}
dtClusterMaxStationRatio = {}
dtClusterMet_dPhi = {}

deltaEta = {}        
deltaRCluster = {}

metPhi = {}
evtNum = {}
runNum = {}
lumiNum = {}

nMe11 = {}

tree_keys = []
nClusterRatio = {}

ME11_veto = {}
ME12_veto = {}
MB1_veto = {}

#L1 plateau
first_in_ME11 = {}
first_in_ME12 = {}
first_in_ME13 = {}
first_in_ME21 = {}
first_in_ME22 = {}
first_in_ME31 = {}
first_in_ME32 = {}
first_in_ME41 = {}
first_in_ME42 = {}


first_in_plateau_ME11 = {}
first_in_plateau_ME12 = {}
first_in_plateau_ME13 = {}
first_in_plateau_ME21 = {}
first_in_plateau_ME22 = {}
first_in_plateau_ME31 = {}
first_in_plateau_ME32 = {}
first_in_plateau_ME41 = {}
first_in_plateau_ME42 = {}
first_in_plateau = {}

#Angular variables
dPhi_csc_csc = {}
dEta_csc_csc = {}
dR_csc_csc = {}
dt_csc_csc = {}
dPhi_csc_dt = {}
dEta_csc_dt = {}
dR_csc_dt = {}

## Settings

In [186]:
category = 0#-1#0
category = 1
#-1: at least 1csc
#0: exactly 1 csc
#1: 2csc,
#2: 1csc+1dt, 

do_jet_veto = False
do_inverted_jet_veto = False
if (do_jet_veto and do_inverted_jet_veto):
    print("Please check jet veto! Aborting...")
    exit()

do_muon_veto = False
do_inverted_muon_veto = False
if (do_muon_veto and do_inverted_muon_veto):
    print("Please check muon veto! Aborting...")
    exit()

do_csc_time_spread_cut = False
do_csc_in_time_cut = False
do_ME_veto = False
do_MB_veto = True
do_HLT_eta_cut = False#True
do_L1_plateau = True

plot_string = ""
#plot_string = "_jet_veto"
#plot_string = "_csc_time_spread_cut"
#plot_string = "_csc_in_time_cut"
#plot_string = ""
#plot_string = "_ME_veto"
#plot_string = "_ME_veto_HLT_eta_cut"
#plot_string = "_ME_veto_HLT_eta_cut_at_least_1_cluster"
#plot_string = "_ME_veto_HLT_eta_cut_in_time_cut_at_least_1_cluster"
#plot_string+= "_exactly_1_csc_cluster"

#plot_string+= "_exactly_2_csc_clusters_ME_veto"
#plot_string+= "_exactly_2_csc_clusters_ME_veto_time_spread_cut_jet_muon_veto_2nd"
#plot_string+= "_exactly_2_csc_clusters_time_spread_cut_jet_muon_veto_2nd"
#plot_string+= "_2_csc_clusters_ME_veto"
#plot_string+= "_at_least_1_csc_cluster_ME_veto"
#plot_string+= "_csc_dt_cluster_ME_veto"
#plot_string+= "_csc_dt_cluster_ME_MB_veto"

In [200]:
for k, T in tree.items():
    tree_keys.append(k)
    #if not k == 'data' and not k[-4:] == '1000':continue
########### SELECTION: CLUSTERS ############
    jet_veto_csc = T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30
    muon_veto_csc = np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal')))
    
    sel_csccluster = T.array('cscRechitClusterSize') >= 0
    
    if do_jet_veto:
        sel_csccluster = jet_veto_csc
    if do_inverted_jet_veto:
        sel_csccluster = np.logical_not(jet_veto_csc)       

    if do_muon_veto:
        sel_csccluster = np.logical_and(sel_csccluster, muon_veto_csc)
    if do_inverted_muon_veto:
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(muon_veto_csc))    
          
    #Missing branch:
    #sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<1.2)

    #Missing branch:
    #sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')<1)

    #Time spread cut
    if do_csc_time_spread_cut:
        sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20)
    
    #Time cut
    if do_csc_in_time_cut:
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(
            T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')< 12.5, 
            T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5))

    #ME11 and ME12 veto
    ME11_veto[k] = np.logical_and(
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11')[:,0]==0  ,
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11')[:,0]==0
    )  

    ME12_veto[k] = np.logical_and(
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12')[:,0]==0  ,
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12')[:,0]==0
    )      

    print(ME11_veto[k].sum())
    print( ME11_veto[k][ ME11_veto[k].sum()>=1][:,0] )
    print( ME11_veto[k][ ME11_veto[k].sum()>=1][:] )
       
    
    if do_ME_veto:
        sel_csccluster = np.logical_and(sel_csccluster,
                                    np.logical_and(ME11_veto[k],ME12_veto[k])
                                   ) 
    if do_HLT_eta_cut:
        sel_csccluster = np.logical_and(sel_csccluster,sel_trgCluster)
        
        
    sel_dtcluster = T.array('dtRechitClusterSize') >= 0
    #Missing branch:
    #sel_dtcluster = np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1
    jet_veto_dt = np.abs(T.array('dtRechitClusterJetVetoPt')) < 50
    muon_veto_dt = np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId')))

    if do_muon_veto:
        sel_dtcluster = np.logical_and(sel_dtcluster, muon_veto_dt)
    if do_jet_veto:
        sel_dtcluster = np.logical_and(sel_dtcluster, jet_veto_dt)

    MB1_veto[k] = T.array('dtRechitClusterNHitStation1') == 0
    if do_MB_veto:
        sel_dtcluster = np.logical_and(sel_dtcluster, MB1_veto[k])
        
    
########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()    
    HLT_CscCluster_Loose = hlt[:,566]
    HLT_CscCluster_Medium = hlt[:,567]
    HLT_CscCluster_Tight = hlt[:,568]
    HLT_L1CSCShower_DTCluster50 = hlt[:,569]
    HLT_L1CSCShower_DTCluster75 = hlt[:,570]

    #Or of the triggers: starting with the CSC ones
    #hlt_sel_ev[k] = np.logical_or(HLT_CscCluster_Loose,np.logical_or(HLT_CscCluster_Medium,HLT_CscCluster_Tight))
    if category==2:
        hlt_sel_ev[k] = HLT_L1CSCShower_DTCluster50
    else:
        hlt_sel_ev[k] = HLT_CscCluster_Loose
     
    
    #sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == 0)
    #sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    sel_ev[k] = T.array('nLeptons') == 0
    
    sel_ev[k] = (sel_ev[k] & (T.array('evtNum')==9430 ))
    
    #If trigger:
    #We don't have correct trigger emulation in signal at the moment
    if k == 'data': sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    if k == 'data': sel_ev[k] = np.logical_and(sel_ev[k], T.array('runNum')>=360019)

    #At least one pre-selected csc cluster, always required
    sel_ev[k] = np.logical_and(sel_ev[k],sel_csccluster.sum()>=1)
    print("Looking for survivors pre category")
    print(sel_ev[k][ sel_ev[k] ])
    print("n clusters")
    print(sel_csccluster[ sel_ev[k] ].sum())
    
    
    if category == 0:
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    elif category == 1:
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() >= 2)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    elif category == 2:      
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    elif category == -1:      
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() >= 1)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])

    print("Looking for survivors after category")
    print(sel_ev[k][ sel_ev[k] ])
    print("n clusters")
    print(sel_csccluster[ sel_ev[k] ].sum())
########### BRANCHES ############

    
    dtClusterMuonVetoPt[k] = T.array('dtRechitClusterMuonVetoPt')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterJetVetoPt[k] = T.array('dtRechitClusterJetVetoPt')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterPhi[k] = T.array('dtRechitClusterPhi')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterEta[k] = T.array('dtRechitClusterEta')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterSize[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterTime[k] =  T.array('dtRechitCluster_match_RPCBx_dPhi0p5')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterNStation[k] =  T.array('dtRechitClusterNStation10')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterNHitStation1[k] = T.array('dtRechitClusterNHitStation1')[sel_dtcluster][sel_ev[k]][:,:]
    dtCluster_match_MB1hits_0p4[k] = T.array('dtRechitCluster_match_MB1hits_0p4')[sel_dtcluster][sel_ev[k]][:,:]
    dtCluster_match_MB1hits_0p5[k] = T.array('dtRechitCluster_match_MB1hits_0p5')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterMet_dPhi[k] =  T.array('dtRechitCluster' + cluster_index + 'Met_dPhi')[sel_dtcluster][sel_ev[k]][:,:]
    dtClusterMaxStationRatio[k] = T.array('dtRechitCluster' + cluster_index + 'MaxStationRatio')[sel_dtcluster][sel_ev[k]][:,:]
        
    cscClusterMuonVetoPt[k] = T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterJetVetoPt[k] = T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterPhi[k] = T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterR[k] = np.sqrt(  T.array('cscRechitCluster' + cluster_index + 'X')[sel_csccluster][sel_ev[k]][:,:]**2 + T.array('cscRechitCluster' + cluster_index + 'Y')[sel_csccluster][sel_ev[k]][:,:]**2 )
    cscClusterZ[k] = T.array('cscRechitCluster' + cluster_index + 'Z')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterTime[k] =  T.array('cscRechitCluster' + cluster_index + 'Time')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterTimeSpread[k] =  T.array('cscRechitCluster' + cluster_index + 'TimeSpread')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterNStation[k] =  T.array('cscRechitCluster' + cluster_index + 'NStation10')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterMet_dPhi[k] =  T.array('cscRechitCluster' + cluster_index + 'Met_dPhi')[sel_csccluster][sel_ev[k]][:,:]
    nCscClusters[k] = sel_csccluster.sum()[sel_ev[k]]
    nDtClusters[k] = sel_dtcluster.sum()[sel_ev[k]]
    
    cscClusterNRechitMinus11[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterNRechitPlus11[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterNRechitMinus12[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12')[sel_csccluster][sel_ev[k]][:,:]
    cscClusterNRechitPlus12[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12')[sel_csccluster][sel_ev[k]][:,:]

      
    sel_jetveto_csc[k] = jet_veto_csc[sel_csccluster][sel_ev[k]][:,:]
    sel_muonveto_csc[k] = muon_veto_csc[sel_csccluster][sel_ev[k]][:,:]
       
    sel_jetveto_dt[k] = jet_veto_dt[sel_dtcluster][sel_ev[k]][:,:]
    sel_muonveto_dt[k] = muon_veto_dt[sel_dtcluster][sel_ev[k]][:,:]        

    dtClusterMaxStation[k] = T.array('dtRechitClusterMaxStation')[sel_dtcluster][sel_ev[k]]
       
    hlt_sel_ev[k] = hlt_sel_ev[k][ sel_ev[k] ]
    weight[k] = T.array('weight')[ sel_ev[k] ]
    evtNum[k] = T.array('evtNum')[ sel_ev[k] ]
    sel_ev[k] = sel_ev[k][ sel_ev[k] ]
    
    all_weight[k] = T.array('weight')

    nClusterRatio[k] = np.divide(nCscClusters[k].astype(float),nDtClusters[k].astype(float),out=np.zeros_like(nDtClusters[k].astype(float)),where=(nDtClusters[k].astype(float) != 0))

IndexError: index 0 is out of bounds for jagged min size 0

* Study delta phi with met
* Apply at least one csc cluster requirement
* Apply L1 plateau

# L1 plateau

In [183]:
first_in_tr1 = {}
first_in_tr2 = {}
first_in_tr3 = {}

sel_ev_pl = {}

for k in tree_keys:
    first_in_ME11[k] = (cscClusterR[k][:,0]>100)&(cscClusterR[k][:,0]<275) &(abs(cscClusterZ[k][:,0])>580)&(abs(cscClusterZ[k][:,0])<632) 
    first_in_ME12[k] = (cscClusterR[k][:,0]>275)&(cscClusterR[k][:,0]<465) &(abs(cscClusterZ[k][:,0])>668)&(abs(cscClusterZ[k][:,0])<724)
    first_in_ME13[k] = (cscClusterR[k][:,0]>505)&(cscClusterR[k][:,0]<700) &(abs(cscClusterZ[k][:,0])>668)&(abs(cscClusterZ[k][:,0])<724)

    first_in_ME21[k] = (cscClusterR[k][:,0]>139)&(cscClusterR[k][:,0]<345) &(abs(cscClusterZ[k][:,0])>789)&(abs(cscClusterZ[k][:,0])<850)
    first_in_ME22[k] = (cscClusterR[k][:,0]>357)&(cscClusterR[k][:,0]<700) &(abs(cscClusterZ[k][:,0])>791)&(abs(cscClusterZ[k][:,0])<850)

    first_in_ME31[k] = (cscClusterR[k][:,0]>160)&(cscClusterR[k][:,0]<345) &(abs(cscClusterZ[k][:,0])>915)&(abs(cscClusterZ[k][:,0])<970)
    first_in_ME32[k] = (cscClusterR[k][:,0]>357)&(cscClusterR[k][:,0]<700) &(abs(cscClusterZ[k][:,0])>911)&(abs(cscClusterZ[k][:,0])<970)

    first_in_ME41[k] = (cscClusterR[k][:,0]>178)&(cscClusterR[k][:,0]<345) &(abs(cscClusterZ[k][:,0])>1002)&(abs(cscClusterZ[k][:,0])<1063)
    first_in_ME42[k] = (cscClusterR[k][:,0]>357)&(cscClusterR[k][:,0]<700) &(abs(cscClusterZ[k][:,0])>1002)&(abs(cscClusterZ[k][:,0])<1063)
    
    first_in_plateau_ME11[k] = first_in_ME11[k] & (cscClusterSize[k][:,0]>=500)
    first_in_plateau_ME21[k] = first_in_ME21[k] & (cscClusterSize[k][:,0]>=500)
    first_in_plateau_ME31[k] = first_in_ME31[k] & (cscClusterSize[k][:,0]>=500)
    first_in_plateau_ME41[k] = first_in_ME41[k] & (cscClusterSize[k][:,0]>=500)

    first_in_plateau_ME12[k] = first_in_ME12[k] & (cscClusterSize[k][:,0]>=200)
    first_in_plateau_ME13[k] = first_in_ME13[k] & (cscClusterSize[k][:,0]>=200)
    first_in_plateau_ME22[k] = first_in_ME22[k] & (cscClusterSize[k][:,0]>=200)
    first_in_plateau_ME32[k] = first_in_ME32[k] & (cscClusterSize[k][:,0]>=200)
    first_in_plateau_ME42[k] = first_in_ME42[k] & (cscClusterSize[k][:,0]>=200)    
    
    first_in_plateau[k] = first_in_plateau_ME11[k] | first_in_plateau_ME12[k] | first_in_plateau_ME13[k] | first_in_plateau_ME21[k] | first_in_plateau_ME22[k] | first_in_plateau_ME31[k] | first_in_plateau_ME32[k] | first_in_plateau_ME41[k] | first_in_plateau_ME42[k]
    
    print(first_in_plateau[k])

    first_in_tr1[k] = np.logical_and( cscClusterSize[k][:,0] >= 100, np.logical_and(cscClusterNStation[k][:,0]>=2, np.abs(cscClusterEta[k][:,0])<1.9))
    first_in_tr2[k] = np.logical_and( cscClusterSize[k][:,0] >= 200, np.logical_and(cscClusterNStation[k][:,0]==1, np.abs(cscClusterEta[k][:,0])<1.9))
    first_in_tr3[k] = np.logical_and( cscClusterSize[k][:,0] >= 500, np.abs(cscClusterEta[k][:,0])>=1.9)
 
    ME1_veto_generic = ((cscClusterNRechitMinus11[k]==0) & (cscClusterNRechitPlus11[k]==0) & (cscClusterNRechitMinus12[k]==0) & (cscClusterNRechitPlus12[k]==0))#.all()
    ME1_veto_cl_0 = (cscClusterNRechitMinus11[k][:,0]==0) & (cscClusterNRechitPlus11[k][:,0]==0) & (cscClusterNRechitMinus12[k][:,0]==0) & (cscClusterNRechitPlus12[k][:,0]==0)
    ME1_veto_cl_1 = (cscClusterNRechitMinus11[k][:,1]==0) & (cscClusterNRechitPlus11[k][:,1]==0) & (cscClusterNRechitMinus12[k][:,1]==0) & (cscClusterNRechitPlus12[k][:,1]==0)
   
    print(ME1_veto_generic.sum())
    sel_ev_pl[k] = np.logical_and(sel_ev[k],first_in_plateau[k])# & ME1_veto_generic#ME1_veto_cl_0 & ME1_veto_cl_1
    
    #since I have removed n clusters==2 above (to avoid blocking events with 3 clusters), need to apply it now 
    if category==1:
        dPhi_csc_csc[k] = ( deltaPhi(  cscClusterPhi[k][:,0].flatten(), cscClusterPhi[k][:,1].flatten() ) )
        dEta_csc_csc[k] = (  cscClusterEta[k][:,0].flatten() - cscClusterEta[k][:,1].flatten() )
        dR_csc_csc[k]   = deltaR( cscClusterEta[k][:,0].flatten(), cscClusterPhi[k][:,0].flatten(), cscClusterEta[k][:,1].flatten(), cscClusterPhi[k][:,1].flatten() )
        dt_csc_csc[k]   = (  cscClusterTime[k][:,0].flatten() - cscClusterTime[k][:,1].flatten() )
        min_dphi = 1.8#.65
        sel_ev_post[k] = np.logical_and(sel_ev_pl[k],np.abs(dPhi_csc_csc[k])>min_dphi)
    if category==2:
        dPhi_csc_dt[k] = ( deltaPhi(  cscClusterPhi[k][:,0].flatten(), dtClusterPhi[k][:,0].flatten() ) )
        dEta_csc_dt[k] = (  cscClusterEta[k][:,0].flatten() - dtClusterEta[k][:,0].flatten() )
        dR_csc_dt[k]   = deltaR( cscClusterEta[k][:,0].flatten(), cscClusterPhi[k][:,0].flatten(), dtClusterEta[k][:,0].flatten(), dtClusterPhi[k][:,0].flatten() )
        sel_ev_post[k] = sel_ev_pl[k]

[ True]
[2]


## Debug yields

In [161]:
lumi=1
weight['signal'] = weight['signal'].astype(bool)

print("Selection \t Yield \t\t Eff. vs no cuts(%)")

table_label = "No cuts"
n_val = np.sum(all_weight['signal'])*lumi
print("%s\t\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))

table_label = "2 CSC + 0 lep"
n_val = (np.sum(weight['signal'])*lumi)
print("%s\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))

table_label = "Trigger plateau"
n_val = (np.sum(weight['signal'][ sel_ev_pl['signal']  ])*lumi)
print("%s\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))

table_label = "Min dphi"
n_val = (np.sum(weight['signal'][ sel_ev_post['signal']  ])*lumi)
print("%s\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))

ME1_veto_cl_0 = (cscClusterNRechitMinus11[k][:,0]==0) & (cscClusterNRechitPlus11[k][:,0]==0) & (cscClusterNRechitMinus12[k][:,0]==0) & (cscClusterNRechitPlus12[k][:,0]==0)
ME1_veto_cl_1 = (cscClusterNRechitMinus11[k][:,1]==0) & (cscClusterNRechitPlus11[k][:,1]==0) & (cscClusterNRechitMinus12[k][:,1]==0) & (cscClusterNRechitPlus12[k][:,1]==0)


table_label = "+ME1[0] veto"
n_val = (np.sum(weight['signal'][ sel_ev_post['signal'] & ME1_veto_cl_0  ])*lumi)
print("%s\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))


table_label = "+ME1[1] veto"
n_val = (np.sum(weight['signal'][ sel_ev_post['signal'] & ME1_veto_cl_1  ])*lumi)
print("%s\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))

table_label = "+ME1[01] veto"
n_val = (np.sum(weight['signal'][ sel_ev_post['signal'] & ME1_veto_cl_0  & ME1_veto_cl_1  ])*lumi)
print("%s\t %.2f\t %.3f" % (table_label, n_val, (100*n_val)/signal_gen_yield))

print(evtNum['signal'][ sel_ev_post['signal'] & ME1_veto_cl_0  & ME1_veto_cl_1  ])

Selection 	 Yield 		 Eff. vs no cuts(%)
No cuts		 48.58	 0.004
2 CSC + 0 lep	 0.00	 0.000
Trigger plateau	 0.00	 0.000
Min dphi	 0.00	 0.000
+ME1[0] veto	 0.00	 0.000
+ME1[1] veto	 0.00	 0.000
+ME1[01] veto	 0.00	 0.000
[]


* Look only at 10 k entries and find the issue

Old method

New

Event 9430 is discarded by the latter method --> inspect!

In [147]:
cut = sel_ev_post['signal'] & ME1_veto_cl_0  & ME1_veto_cl_1 & (evtNum['signal'] == 9430)

print()

Previous ME1 veto in the first loop

ME1 veto done after the trigger plateau with surviving clusters is different for some reasons

If I include ME veto in the trigger plateau, I get the same as the independent computation

* Tune min dphi cut to remove poorly-clustered events

# ABCD

In [9]:
PHI_MIN = 1.8
PHI_CUT = 2.85
#PHI_MAX = 2.
PHI_MAX = 3.15

N_MIN = 50
N_MAX = 500000000
N_CUT = 240

n_ev = 5000

pre_cut = {}
a = {}
b = {}
c = {}
d = {}

In [10]:
def return_abcd_masks(var1,var2,MIN1,CUT1,MAX1,MIN2,CUT2,MAX2,sel):
    d = sel & (var1>=CUT1) & (var1<MAX1) & ( var2>MIN2 ) & ( var2<CUT2 )
    b = sel & (var1>=MIN1) & (var1<CUT1) & ( var2>=CUT2 ) & ( var2<MAX2 ) 
    c = sel & (var1>=MIN1) & (var1<CUT1) & ( var2>MIN2 ) & ( var2<CUT2 )  
    a = sel & (var1>=CUT1) & (var1<MAX1) & ( var2>=CUT2 ) & ( var2<MAX2 )
    return a,b,c,d

## SR background levels

* Scan minimum delta phi threshold
* Scan optimal delta phi cut

## Scan phi cut

In [11]:
#0, 0.5, 0.8, 1, 1.1, 1.2
dr_min_list = [0.5]#[0.9]#[1.9]
phi_min_scan_list = [0,0.2,0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.9,0.95,1.,1.05,1.1,1.15,1.2,1.25,1.3,1.35,1.4,1.45,1.5]

phi_min_scan_list = [1.1]
phi_cut_scan_list = np.linspace(1.5,3.1,64+1)#[1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,3.]
phi_cut_scan_list = np.linspace(1.5,3.1,32+1)

phi_cut_scan_list = [2.85]

time_cut_string = "IT"
plot_string_plus = plot_string+"_"+time_cut_string+"_dR"+str( dr_min_list[0] ).replace(".","p")
leghead = "|#Delta#eta|<1.9, "+time_cut_string+", #DeltaR>"+str(dr_min_list[0])+","

#one per min dphi cut
bkg_rate = {}
#bkg_unc = {}
observation = {}
signal_rate = {}

normalization = 1

for phi_cut_scan in phi_cut_scan_list:
    signal_rate[phi_cut_scan] = {}

print('k\t phi\t B\t C\t D\t A\t pred\t\t zval\t perc')
if category==1:
    #Signal
    #Data
    for k in ['signal']:
        tmp_dphi = np.abs( dPhi_csc_csc[k] )
        tmp_deta = np.abs(  dEta_csc_csc[k] )
        tmp_dR = np.abs(  dR_csc_csc[k] )
        tmp_dt = np.abs( dt_csc_csc[k] )
        var = cscClusterSize[k][:,1]
        t_var = cscClusterTime[k][:,1]
        t_spread_var = cscClusterTimeSpread[k][:,1]
        dphi = np.abs(dPhi_csc_csc[k])
        tmp_cut = sel_ev_post[k]  & (np.abs(tmp_deta)<1.9) & (np.abs(t_var)<15) & (np.abs(t_spread_var)<20) & (sel_jetveto_csc[k][:,1]==True) & (sel_muonveto_csc[k][:,1]==True)#& (t_var<-15) #& (t_var<-15) # #& (tmp_dR>r) #(tmp_deta>=1.9)  & ( t_var<-15  )
        for phi_cut_scan in phi_cut_scan_list:
            if phi_cut_scan<=PHI_MIN: continue
            if 'data' in k:
                a[k],b[k],c[k],d[k],pred,unc_pred, p_value, z_value = run_abcd(
                    var,dphi,
                    N_MIN,N_CUT,N_MAX,
                    PHI_MIN,phi_cut_scan,PHI_MAX,
                    tmp_cut
                    )
                print("%s\t %.2f\t %i\t %i\t %i\t %i\t %.2f +- %.2f\t %.2f\t %.2f" % (k,phi_cut_scan,b[k],c[k],d[k],a[k],pred,unc_pred,z_value, 100*(pred-a[k])/a[k]))
                observation[phi_cut_scan] = [ pred,b[k],c[k],d[k] ]
                bkg_rate[phi_cut_scan] = [ pred,b[k],c[k],d[k] ]
                #bkg_unc[phi_min_scan] = [ unc_pred,np.sqrt(b[k]),np.sqrt(c[k]),np.sqrt(d[k]) ]
            else:
                ma,mb,mc,md = return_abcd_masks(
                    var,dphi,
                    N_MIN,N_CUT,N_MAX,
                    PHI_MIN,phi_cut_scan,PHI_MAX,
                    tmp_cut
                    )
                a[k] = (np.sum(weight[k][ ma ]))*lumi
                b[k] = (np.sum(weight[k][ mb ]))*lumi
                c[k] = (np.sum(weight[k][ mc ]))*lumi
                d[k] = (np.sum(weight[k][ md ]))*lumi
                print(k,phi_cut_scan,a[k],b[k],c[k],d[k], a[k]+b[k]+c[k]+d[k])
                signal_rate[phi_cut_scan]['ggH'] = [ a[k]*normalization,b[k]*normalization,c[k]*normalization,d[k]*normalization ]



k	 phi	 B	 C	 D	 A	 pred		 zval	 perc
signal 2.85 376.3480396568775 590.7999835163355 520.1214528828859 289.96317841112614 1777.232654467225


In [12]:
#print(ME11_veto['signal'])
print("Selection \t Yield \t\t Eff. vs no cuts(%)")
print("%s\t\t %.2f\t %.3f" % ("No cuts",(np.sum(all_weight['signal'])*lumi) , (100*np.sum(all_weight['signal'])*lumi)/signal_gen_yield))
#print( (100*np.sum(all_weight['signal'][ ME11_veto['signal'][:,0]==True  ])*lumi)/signal_gen_yield)

print("%s\t %.2f\t %.3f" % ("2 CSC + 0 lep", (np.sum(weight['signal'])*lumi) , (100*np.sum(weight['signal'])*lumi)/signal_gen_yield))

ME1_veto_cl_0 = (cscClusterNRechitMinus11[k][:,0]==0) & (cscClusterNRechitPlus11[k][:,0]==0) & (cscClusterNRechitMinus12[k][:,0]==0) & (cscClusterNRechitPlus12[k][:,0]==0)

ME1_veto_cl_1 = (cscClusterNRechitMinus11[k][:,1]==0) & (cscClusterNRechitPlus11[k][:,1]==0) & (cscClusterNRechitMinus12[k][:,1]==0) & (cscClusterNRechitPlus12[k][:,1]==0)


tmp_cut = ME1_veto_cl_0
print("%s\t %.2f\t %.3f" % ("ME1 veto [0]", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))

tmp_cut = ME1_veto_cl_1
print("%s\t %.2f\t %.3f" % ("ME1 veto [1]", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))

tmp_cut = ME1_veto_cl_1 & ME1_veto_cl_0
print("%s\t %.2f\t %.3f" % ("ME1 veto [0&1]", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))


print("%s\t %.2f\t %.3f" % ("Trigger plateau", (np.sum(weight['signal'][ sel_ev['signal']  ])*lumi), (100*np.sum(weight['signal'][ sel_ev['signal'] ])*lumi)/signal_gen_yield))

print("%s\t %.2f\t %.3f" % ("Trigger + ME1[0]", (np.sum(weight['signal'][ sel_ev['signal'] & ME1_veto_cl_0 ])*lumi), (100*np.sum(weight['signal'][ sel_ev['signal'] & ME1_veto_cl_0 ])*lumi)/signal_gen_yield))
print("%s\t %.2f\t %.3f" % ("Trigger + ME1[0&1]", (np.sum(weight['signal'][ sel_ev['signal'] & ME1_veto_cl_0 & ME1_veto_cl_1 ])*lumi), (100*np.sum(weight['signal'][ sel_ev['signal'] & ME1_veto_cl_0 & ME1_veto_cl_1 ])*lumi)/signal_gen_yield))

print("%s\t %.2f\t %.3f" % ("Min dphi>1.8", (np.sum(weight['signal'][ sel_ev_post['signal'] ])*lumi), (100*np.sum(weight['signal'][ sel_ev_post['signal'] ])*lumi)/signal_gen_yield))



k='signal'
tmp_dphi = np.abs( dPhi_csc_csc[k] )
tmp_deta = np.abs(  dEta_csc_csc[k] )
var = cscClusterSize[k][:,1]
t_var = cscClusterTime[k][:,1]
t_spread_var = cscClusterTimeSpread[k][:,1]



dphi = np.abs(dPhi_csc_csc[k])
tmp_cut = sel_ev_post[k]  & (np.abs(tmp_deta)<1.9)# & (np.abs(t_var)<15) & (np.abs(t_spread_var)<20) & (sel_jetveto_csc[k][:,1]==True) & (sel_muonveto_csc[k][:,1]==True)#& (t_var<-15) #& (t_var<-15) # #& (tmp_dR>r) #(tmp_deta>=1.9)  & ( t_var<-15  )
print("%s\t %.2f\t %.3f" % ("Delta eta<1.9", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))
tmp_cut = sel_ev_post[k]  & (np.abs(tmp_deta)<1.9) & (np.abs(t_var)<15)# & (np.abs(t_spread_var)<20) & (sel_jetveto_csc[k][:,1]==True) & (sel_muonveto_csc[k][:,1]==True)#& (t_var<-15) #& (t_var<-15) # #& (tmp_dR>r) #(tmp_deta>=1.9)  & ( t_var<-15  )
print("%s\t %.2f\t %.3f" % ("In time cut", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))
tmp_cut = sel_ev_post[k]  & (np.abs(tmp_deta)<1.9) & (np.abs(t_var)<15) & (np.abs(t_spread_var)<20)# & (sel_jetveto_csc[k][:,1]==True) & (sel_muonveto_csc[k][:,1]==True)#& (t_var<-15) #& (t_var<-15) # #& (tmp_dR>r) #(tmp_deta>=1.9)  & ( t_var<-15  )
print("%s\t %.2f\t %.3f" % ("Time spread", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))
tmp_cut = sel_ev_post[k]  & (np.abs(tmp_deta)<1.9) & (np.abs(t_var)<15) & (np.abs(t_spread_var)<20) & (sel_jetveto_csc[k][:,1]==True) & (sel_muonveto_csc[k][:,1]==True)#& (t_var<-15) #& (t_var<-15) # #& (tmp_dR>r) #(tmp_deta>=1.9)  & ( t_var<-15  )
print("%s\t %.2f\t %.3f" % ("Jet muon vetoes", (np.sum(weight['signal'][ tmp_cut ])*lumi), (100*np.sum(weight['signal'][ tmp_cut ])*lumi)/signal_gen_yield))




ma,mb,mc,md = return_abcd_masks(
                        var,dphi,
                        N_MIN,N_CUT,N_MAX,
                        PHI_MIN,2.85,PHI_MAX,
                        tmp_cut
                        )
#all_abcd = np.array(ma or mb or mc or md).flatten()
#ma = np.array(ma).flatten()
#mb = np.array(mb).flatten()
#mc = np.array(mc).flatten()
#md = np.array(md).flatten()

MIN1 = N_MIN
CUT1 = N_CUT
MAX1 = N_MAX
MIN2 = PHI_MIN
CUT2 = 2.85
MAX2 = PHI_MAX
#print(MIN1,CUT1,MAX1)
#print(MIN2,CUT2,MAX2)
manual_md = tmp_cut & (var>=CUT1) & (var<MAX1) & ( dphi>MIN2 ) & ( dphi<CUT2 )#.astype(bool)
manual_mb = tmp_cut & (var>=MIN1) & (var<CUT1) & ( dphi>=CUT2 ) & ( dphi<MAX2 )#.astype(bool)
manual_mc = tmp_cut & (var>=MIN1) & (var<CUT1) & ( dphi>MIN2 ) & ( dphi<CUT2 )#.astype(bool)
manual_ma = tmp_cut & (var>=CUT1) & (var<MAX1) & ( dphi>=CUT2 ) & ( dphi<MAX2 )#.astype(bool)

all_abcd = tmp_cut & (manual_ma | manual_mb | manual_mc | manual_md)
tmp_cut2 = tmp_cut & all_abcd
print("%s\t %.2f\t %.2f" % ("ABCD plane", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
tmp_cut2 = tmp_cut & manual_mc
print("%s\t %.2f\t %.2f" % ("C", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
tmp_cut2 = tmp_cut & manual_mb
print("%s\t %.2f\t %.2f" % ("B", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
tmp_cut2 = tmp_cut & manual_md
print("%s\t %.2f\t %.2f" % ("D", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
tmp_cut2 = tmp_cut & manual_ma
print("%s\t %.2f\t %.2f" % ("A", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))

#print("-----")
#all_abcd = tmp_cut & (ma | mb | mc | md)
#tmp_cut2 = tmp_cut & all_abcd
#print("%s\t %.2f\t %.2f" % ("ABCD plane", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
#tmp_cut2 = tmp_cut & mc
#print("%s\t %.2f\t %.2f" % ("C", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
#tmp_cut2 = tmp_cut & mb
#print("%s\t %.2f\t %.2f" % ("B", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
#tmp_cut2 = tmp_cut & md
#print("%s\t %.2f\t %.2f" % ("D", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))
#tmp_cut2 = tmp_cut & ma
#print("%s\t %.2f\t %.2f" % ("A", (np.sum(weight['signal'][ tmp_cut2 ])*lumi), (100*np.sum(weight['signal'][ tmp_cut2 ])*lumi)/signal_gen_yield))


Selection 	 Yield 		 Eff. vs no cuts(%)
No cuts		 1118331.31	 100.000
2 CSC + 0 lep	 15543.23	 1.390
ME1 veto [0]	 7406.14	 0.662
ME1 veto [1]	 4898.57	 0.438
ME1 veto [0&1]	 2461.67	 0.220
Trigger plateau	 15543.23	 1.390
Trigger + ME1[0]	 7406.14	 0.662
Trigger + ME1[0&1]	 2461.67	 0.220
Min dphi>1.8	 2795.12	 0.250
Delta eta<1.9	 2474.96	 0.221
In time cut	 2341.45	 0.209
Time spread	 2230.30	 0.199
Jet muon vetoes	 1777.23	 0.159
ABCD plane	 1777.23	 0.16
C	 520.12	 0.05
B	 590.80	 0.05
D	 289.96	 0.03
A	 376.35	 0.03
